In [2]:
import ktrain
from ktrain import text
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1' #To ignore GPU

In [4]:
#We use 15% of the train data for validation
train, val, preproc = text.texts_from_folder('../datasets/ChnSentiCorp_htl_ba_6000', 
                                            maxlen=75, 
                                            max_features=30000,
                                            preprocess_mode='bert',
                                            train_test_names=['train'],
                                            val_pct=0.15,
                                            classes=['pos', 'neg'])

detected encoding: GB18030
Decoding with GB18030 failed 1st attempt - using GB18030 with skips
skipped 104 lines (0.3%) due to character decoding errors
skipped 14 lines (0.2%) due to character decoding errors
downloading pretrained BERT model (chinese_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: zh-cn


preprocessing test...
language: zh-cn


In [5]:
model = text.text_classifier('bert', train, preproc=preproc)
learner = ktrain.get_learner(model, 
                             train_data=train, 
                             val_data=val, 
                             batch_size=32)

Is Multi-Label? False
maxlen is 75
done.


In [6]:
learner.fit_onecycle(2e-5, 4, checkpoint_folder='/tmp/saved_weights')



begin training using onecycle policy with max lr of 2e-05...
Train on 5028 samples, validate on 888 samples
Epoch 1/4
5028/5028 [==============================] - 1010s 201ms/sample - loss: 0.4415 - accuracy: 0.7969 - val_loss: 0.2450 - val_accuracy: 0.8953
Epoch 2/4
5028/5028 [==============================] - 997s 198ms/sample - loss: 0.2371 - accuracy: 0.9053 - val_loss: 0.2068 - val_accuracy: 0.9110
Epoch 3/4
5028/5028 [==============================] - 1095s 218ms/sample - loss: 0.1260 - accuracy: 0.9525 - val_loss: 0.2286 - val_accuracy: 0.9212
Epoch 4/4
5028/5028 [==============================] - 1067s 212ms/sample - loss: 0.0441 - accuracy: 0.9855 - val_loss: 0.2181 - val_accuracy: 0.9381


In [7]:
learner.view_top_losses(n=1, preproc=preproc)

----------
id:170 | loss:8.75 | true:pos | pred:neg)

[CLS]设施陈旧，衣柜的推拉门是坏的，电视机较小，碰巧酒店在对有线电视网路整改，所以只能收到4，5个频道，晕！房间隔音效果很差。优点是交通还算方便，到金街[SEP]


In [8]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [9]:
predictor.predict("这家酒店的风景和服务都非常糟糕") #The view and service of this hotel are very bad

'neg'

In [10]:
predictor.predict("我喜欢这家酒店的服务") #I like the service of this hotel

'pos'

In [11]:
predictor.save('/tmp/mypred')
predictor = ktrain.load_predictor('/tmp/mypred')
predictor.predict('我喜欢这家酒店的服务') #I like the service of this hotel

'pos'